In [22]:
import hashlib
import random
from sage.all import *

class EcDSA_Ed25519:
    def __init__(self, p, a, d):
        assert a != d and is_prime(p) and p > 3
        K = GF(p)

        A =  2*(a + d)/(a - d)
        B =  4/(a - d)

        self.alfa = A/(3*B) ; self.s = B

        a4 =  self.s**(-2) - 3*self.alfa**2
        a6 =  -self.alfa**3 - a4*self.alfa

        self.EC = EllipticCurve(K,[a4,a6])

        self.Px = K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.Py = K(46316835694926478169428394003475163141307993866256225615783033603165251855960)

        self.L = ZZ(2**252 + 27742317777372353535851937790883648493)
        self.P = self.ed2ec(self.Px, self.Py)

        self.private_key = self.generate_private_key()

    def generate_private_key(self):
        return randint(1, self.L - 1)
    
    def generate_public_key(self):
        return self.private_key * self.P
    
    def ed2ec(self,x,y):      ## mapeia Ed --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.alfa; s = self.s
        return self.EC(z/s + alfa , w/s)

    def sign(self, message):
        h = hashlib.sha512(message).digest()
        h_int = int.from_bytes(h, 'big') % self.L

        k = randint(1, self.L - 1)
        R = k * self.P

        r = int(R[0]) % self.L

        s = (k + h_int * self.private_key) % self.L

        return (r, s)
    
    def verify(self, message, signature, public_key):
        r, s = signature

        if not (1 <= r < self.L and 1 <= s < self.L):
            return False
        
        if not public_key in self.EC:
            return False
        
        h = hashlib.sha512(message).digest()
        h_int = int.from_bytes(h, 'big') % self.L

        R_prime = s * self.P - h_int * public_key

        return int(R_prime[0]) % self.L == r

if __name__ == "__main__":
    p = 2**255 - 19
    K = GF(p)
    a = K(-1)
    d = K(-121665) / K(121666)

    ecdsa = EcDSA_Ed25519(p, a, d)
    public_key = ecdsa.generate_public_key()
    message = b"Ola!"
    signature = ecdsa.sign(message)
    is_valid = ecdsa.verify(message, signature, public_key)

    print(f"Assinatura válida? {is_valid}")

Assinatura válida? True
